## Integration with PACKIO outputs (.paq data)

PACKIO is a software for temporal synchronization of experimental data acquisition and simultaneous hardware triggering. More information can be found at: https://github.com/apacker83/PackIO

The outputs of PACKIO is a .paq file that contains data across all channels collected.

In [ ]:
import packerlabimaging as pli
prep = 'PS12'
date = '2021-01-25'

expobj: pli.Experiment = pli.import_obj(pkl_path=f'/home/pshah/mnt/qnap/Analysis/{date}/{prep}/{prep}_analysis.pkl')

trialobj = expobj.load_trial(trialID=expobj.trialIDs[0])


#### adding paqdata module to the trialobj

In [ ]:
from packerlabimaging.processing.paq import PaqData

In [ ]:
# adding paqdata to the trialobj

paq_path = '/home/pshah/mnt/qnap/Data/2021-01-25/2021-01-25_PS12_001.paq'
trialobj.Paq, _ = PaqData.import_paqdata(paq_path=paq_path, plot=True)

In [ ]:
# inspect the newly created .Paq module
print(trialobj.Paq)

We can perform a quick plot of paq data channel for quick visualization using the `plot__paq_channel()` function from the `plotting` submodule.

In [ ]:
# plot paq channel data
pli.plotting.plot__paq_channel(trialobj.Paq, channel='voltage', x_axis='Time (secs)', x_tick_secs=120)

#### Processing paq data

In this paq data, one of the channels recorded the microscope imaging frame clock. We can retrieve these frame clock times to retrieve the paq clock timestamps when the imaging period began and finished.

In [ ]:
# finding frame times of imaging from the frame clock channel ("frame_clock").

trialobj.Paq.frame_times = trialobj.Paq.paq_frame_times(frame_channel='frame_clock')


In [ ]:
# crop all channels in paq data to the start and finish of the imaging period within the trial.
trialobj.Paq.cropPaqData(begin=trialobj.Paq.frame_times[0], end=trialobj.Paq.frame_times[1], channels='all')

In [ ]:
# plot to confirm that the cropping executed as expected
pli.plotting.plot__paq_channel(trialobj.Paq, channel='voltage', x_axis='Time (secs)', x_tick_secs=120)

#### Synchronizing .paq data with imaging frames

Since, imaging is generally performed at multiple orders of magnitude slower data collection speed, we have provided the `get_sparse_paq()` method in the .Paq module to collect paq data from channels at timepoints synchronized with only the imaging frames (found under `.Paq.frame_times`.

In [ ]:
trialobj.Paq.get_sparse_paq(frame_clock=trialobj.Paq.frame_times)